In [2]:
# If not installed, run this cell. It is needed for the determination of the recession constant
#!pip install hydrosignatures
#!pip show hydrosignatures

In [3]:
#!pip install pygeohydro
#!pip show pygeohydro

In [ ]:
# The following two lines had to be executed to correct the modifications on library dependencies that the above libraries carried out when installing
#!pip install -U --force-reinstall scipy
#!pip install matplotlib --upgrade
#!pip install aiohttp --upgrade

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

import hydrosignatures as hs
from pygeohydro import NWIS

In [ ]:
''' Example of how to use the baseflow recession function from the hydrosignatures library '''

# Get streamflow data
nwis = NWIS()
q = nwis.get_streamflow("12304500", ("2000-01-01", "2019-12-31"))
mrc_np, bfr_k_np = hs.baseflow_recession(q, fit_method="nonparametric_analytic")
mrc_exp, bfr_k_exp = hs.baseflow_recession(q, fit_method="exponential")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), dpi=100)

ax1.plot(mrc_np[:, 0], mrc_np[:, 1], "bx")
ax1.plot(
    np.sort(mrc_np[:, 0]),
    np.exp(np.log(mrc_np[0, 1]) - bfr_k_np * np.sort(mrc_np[:, 0])),
    "g-",
    linewidth=2,
)
ax1.set_xlabel("Relative time [days]")
ax1.set_ylabel("Flow [cfs]")
ax1.set_xlim(0, 80)
ax1.set_ylim(0, 80)
ax1.set_title("Non-parametric fitted recession curve")
ax1.legend(["MRC", f"Exponential fit (K={bfr_k_np:.4f})"])

ax2.plot(mrc_exp[:, 0], mrc_exp[:, 1], "bx")
ax2.plot(
    np.sort(mrc_exp[:, 0]),
    np.exp(np.log(mrc_exp[0, 1]) - bfr_k_exp * np.sort(mrc_exp[:, 0])),
    "g-",
    linewidth=2,
)
ax2.set_xlabel("Relative time [days]")
ax2.set_ylabel("Flow [cfs]")
ax2.set_xlim(0, 80)
ax2.set_ylim(0, 80)
ax2.set_title("Exponential fitted recession curve")
ax2.legend(["MRC", f"Exponential fit (K={bfr_k_exp:.4f})"])

In [ ]:
''' Comparion of the alpha value obtained from the example (without weights) with the alpha value obtained by using the 'polyfit' function from numpy (with weights) '''

x = list(mrc_np[:, 0])
y = list(mrc_np[:, 1])

# Fit a linear regression model
coefficients = np.polyfit(x, np.log(y), 1, w=np.sqrt(y)) # Use of weights (w) to avoid bias towards small values
beta = np.exp(coefficients[1])
alpha = coefficients[0]
print("The alpha value differs from the example ('bfr_k_np') when a weight is assigned in the 'polyfit' function:", round(alpha, 4))

# Fit a linear regression model
coefficients2 = np.polyfit(x, np.log(y), 1) # Use of weights (w) to avoid bias towards small values
beta2 = np.exp(coefficients2[1])
alpha2 = coefficients2[0]
print("The alpha value is the same value that that from the example ('bfr_k_np') when a weight is not assigned in the 'polyfit' function:", round(alpha2, 4))

In [ ]:
''' Plot of the non-parametric fitted recession curve with and without weights '''

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), dpi=100)

ax1.plot(mrc_np[:, 0], mrc_np[:, 1], "bx")
ax1.plot(
    np.sort(mrc_np[:, 0]),
    np.exp(np.log(mrc_np[0, 1]) - abs(alpha2) * np.sort(mrc_np[:, 0])),
    "g-",
    linewidth=2,
)
ax1.set_xlabel("Relative time [days]")
ax1.set_ylabel("Flow [cfs]")
ax1.set_xlim(0, 80)
ax1.set_ylim(0, 80)
ax1.set_title("Non-parametric fitted recession curve\nWithout weights")
ax1.legend(["MRC", f"Exponential fit (K={abs(alpha2):.4f})"])

ax2.plot(mrc_np[:, 0], mrc_np[:, 1], "bx")
ax2.plot(
    np.sort(mrc_np[:, 0]),
    np.exp(np.log(mrc_np[0, 1]) - abs(alpha) * np.sort(mrc_np[:, 0])),
    "g-",
    linewidth=2,
)
ax2.set_xlabel("Relative time [days]")
ax2.set_ylabel("Flow [cfs]")
ax2.set_xlim(0, 80)
ax2.set_ylim(0, 80)
ax2.set_title("Non-parametric fitted recession curve\n With weights")
ax2.legend(["MRC", f"Exponential fit (K={abs(alpha):.4f})"])

**Coeficiente de determinación ($R^2$)**  
   - Se obtiene elevando al cuadrado el coeficiente de correlación ($R^2$), lo que significa que **no distingue entre correlaciones positivas y negativas**.  
   - Representa el **porcentaje de variabilidad** de una variable que es explicada por la otra en un modelo de regresión.  
   - Si quieres evaluar **qué tan bien un modelo explica la variabilidad de los datos**, este es el mejor indicador.

In [ ]:
''' Coefficient of determination of the non-parametric fitted recession curve with and without weights '''

corr_matrix = np.corrcoef(mrc_np[:, 1], np.exp(np.log(mrc_np[0, 1]) - abs(alpha2) * np.sort(mrc_np[:, 0])))
corr = corr_matrix[0,1]
R_sq = corr**2
print("Coeffcient of determination with alpha value", abs(round(alpha2, 4)), "--without weights-- is", round(R_sq, 3))

corr_matrix2 = np.corrcoef(mrc_np[:, 1], np.exp(np.log(mrc_np[0, 1]) - abs(alpha) * np.sort(mrc_np[:, 0])))
corr2 = corr_matrix2[0,1]
R_sq2 = corr2**2
print("Coeffcient of determination with alpha value", abs(round(alpha, 4)), "--with weights-- is", round(R_sq2, 3))

In [10]:
# Whole-period daily dataset
daily_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Stations\Climate_Sensitive_Stations-GRDC\2025-02-13_17-18_Daily"

# All daily continuos data folder
lcd_folder = daily_folder + "//Baseflow//longest_continuous_data"

In [ ]:
# DF with the longest continuos periods for all the stations
longest_periods_df = pd.read_csv(lcd_folder + "//longest_continuos_df.csv")
longest_periods_df['Station'] = longest_periods_df['Station'].astype(str)
longest_periods_df

In [12]:
# Read the DF with all the daily data
df_cleaned = pd.read_csv(daily_folder + "\_DataFrames\Joined_Daily_Sts_DFs.csv", index_col="YYYY-MM-DD")
df_cleaned.index = pd.to_datetime(df_cleaned.index, format='%Y-%m-%d')
df_cleaned

**"hs.baseflow_recession()"**: Calculate baseflow recession constant and master recession curve.

*Notes*\
This function is ported from the TOSSH Matlab toolbox, which is based on the following publication:

- Gnann, S.J., Coxon, G., Woods, R.A., Howden, N.J.K., McMillan H.K., 2021. TOSSH: A Toolbox for Streamflow Signatures in Hydrology. Environmental Modelling & Software. https://doi.org/10.1016/j.envsoft.2021.104983

This function calculates baseflow recession constant assuming exponential recession behaviour (Safeeq et al., 2013). Master recession curve (MRC) is constructed using the adapted matching strip method (Posavec et al., 2006).

According to Safeeq et al. (2013), K < 0.065 represent groundwater dominated slow-draining systems, K >= 0.065 represent shallow subsurface flow dominated fast draining systems.

*Parameters*
- streamflow : numpy.ndarray\
    Streamflow as a 1D array.

- freq : float, optional\
    Frequency of steamflow in number of days. Default is 1, i.e., daily streamflow.

- recession_length : int, optional\
    Minimum length of recessions [days]. Default is 15.

- n_start : int, optional\
    Days to be removed after start of recession. Default is 0.

- eps : float, optional\
    Allowed increase in flow during recession period. Default is 0.

- start_of_recession : {'baseflow', 'peak'}, optional\
    Define start of recession. Default is 'baseflow'.

- fit_method : {'nonparametric_analytic', 'exponential'}, optional\
    Method to fit mrc. Default is 'nonparametric_analytic'.

- lyne_hollick_smoothing : float, optional\
    Smoothing parameter of Lyne-Hollick filter. Default is 0.925.

*Returns*
- mrc : numpy.ndarray\
    Master Recession Curve as 2D array of [time, flow].

* bf_recession_k : float\
    Baseflow Recession Constant [1/day].

*Raises*
- ValueError\
    If no recession segments are found or if a complex BaseflowRecessionK is calculated.

In [ ]:
''' Determination of alpha factor with all the daily data series '''

# Initialize an empty list to store the results
results = []

for station in df_cleaned.columns:
    print("Processing station:", station)
    
    try:
        # Attempt to calculate alpha without weights
        mrc_np, alpha_without_weights = hs.baseflow_recession(df_cleaned[station], fit_method="nonparametric_analytic")
        
        # Extract x and y values
        x = list(mrc_np[:, 0])
        y = list(mrc_np[:, 1])

        # Calculate alpha with weights
        coefficients = np.polyfit(x, np.log(y), 1, w=np.sqrt(y))
        alpha_with_weights = abs(coefficients[0])

        # Compute R^2 without weights
        corr_matrix_without_weights = np.corrcoef(y, np.exp(np.log(mrc_np[0, 1]) - alpha_without_weights * np.sort(x)))
        corr_without_weights = corr_matrix_without_weights[0, 1]
        R_sq_without_weights = corr_without_weights ** 2

        # Compute R^2 with weights
        corr_matrix_with_weights = np.corrcoef(y, np.exp(np.log(mrc_np[0, 1]) - alpha_with_weights * np.sort(x)))
        corr_with_weights = corr_matrix_with_weights[0, 1]
        R_sq_with_weights = corr_with_weights ** 2
    
    except Exception as e:
        print(f"Error processing station {station}: {e}")
        
        # Assign NaN values if an error occurs
        alpha_without_weights = np.nan
        R_sq_without_weights = np.nan
        alpha_with_weights = np.nan
        R_sq_with_weights = np.nan

    # Store results in a dictionary
    results.append({
        "station": station,
        "alpha_without_weights": alpha_without_weights,
        "R_sq_without_weights": R_sq_without_weights,
        "alpha_with_weights": alpha_with_weights,
        "R_sq_with_weights": R_sq_with_weights
    })

# Convert list of dictionaries into a DataFrame
df_results = pd.DataFrame(results)
df_results

In [14]:
df_results.describe()

In [ ]:
''' Determination of alpha factor with only the longest continuous daily data periods '''

# Initialize an empty list to store the results
results2 = []

for station in df_cleaned.columns:
    print("Processing station:", station)

    # Get the start and end dates of the longest period of each station
    start_date = longest_periods_df[longest_periods_df["Station"] == station]["StartDate"].to_list()[0]
    end_date = longest_periods_df[longest_periods_df["Station"] == station]["EndDate"].to_list()[0]
    
    try:
        # Attempt to calculate alpha without weights
        mrc_np, alpha_without_weights = hs.baseflow_recession(df_cleaned.loc[start_date:end_date, [station]], fit_method="nonparametric_analytic")
        
        # Extract x and y values
        x = list(mrc_np[:, 0])
        y = list(mrc_np[:, 1])

        # Calculate alpha with weights
        coefficients = np.polyfit(x, np.log(y), 1, w=np.sqrt(y))
        alpha_with_weights = abs(coefficients[0])

        # Compute R^2 without weights
        corr_matrix_without_weights = np.corrcoef(y, np.exp(np.log(mrc_np[0, 1]) - alpha_without_weights * np.sort(x)))
        corr_without_weights = corr_matrix_without_weights[0, 1]
        R_sq_without_weights = corr_without_weights ** 2

        # Compute R^2 with weights
        corr_matrix_with_weights = np.corrcoef(y, np.exp(np.log(mrc_np[0, 1]) - alpha_with_weights * np.sort(x)))
        corr_with_weights = corr_matrix_with_weights[0, 1]
        R_sq_with_weights = corr_with_weights ** 2
    
    except Exception as e:
        print(f"Error processing station {station}: {e}")
        
        # Assign NaN values if an error occurs
        alpha_without_weights = np.nan
        R_sq_without_weights = np.nan
        alpha_with_weights = np.nan
        R_sq_with_weights = np.nan

    # Store results in a dictionary
    results2.append({
        "station": station,
        "alpha_without_weights": alpha_without_weights,
        "R_sq_without_weights": R_sq_without_weights,
        "alpha_with_weights": alpha_with_weights,
        "R_sq_with_weights": R_sq_with_weights
    })

# Convert list of dictionaries into a DataFrame
df_results2 = pd.DataFrame(results2)
df_results2

In [16]:
df_results2.describe()

In [17]:
series1 = pd.Series(df_results.describe()["R_sq_with_weights"], name="ALL_DATA") # All daily data series
series2 = pd.Series(df_results2.describe()["R_sq_with_weights"], name="CONT_DATA") # Only the longest continuous daily data periods
# Concatenate the series into a DataFrame for comparison of the statistics of R2 
R_sq_mean = pd.concat([series1, series2], axis=1)
R_sq_mean

Comparison of the **R² with weights** (`R_sq_with_weights`) values for the two datasets:  

- **ALL_DATA**: Statistics resulting from data that includes NaN values from missing data.  
- **CONT_DATA**: Statistics resulting from data that contains only the longest continuous period for each station.  

### Observations from the Comparison:
1. **Count Difference**:  
   - ALL_DATA has **794** valid values, while CONT_DATA has **788**, suggesting that for a bigger number of stations the baseflow alpha factor could not be determined when using the longest continuous period approach.  

2. **Mean & Standard Deviation**:  
   - The mean R² is slightly higher for **ALL_DATA (0.946)** than **CONT_DATA (0.943)**.  
   - The standard deviation is slightly lower for CONT_DATA, suggesting slightly more **consistency** in R² values when using continuous data.  

3. **Minimum Value**:  
   - The lowest R² in **ALL_DATA** is **0.2118**, much lower than the minimum in **CONT_DATA (0.5211)**.  
   - This suggests that **including no-data periods might have led to some very poor model fits** (possibly due to discontinuities in the hydrograph).  

4. **Median (50%) & Quartiles**:  
   - The **median** is nearly identical (**0.9663 for both**), suggesting that for most stations, the effect of using continuous data is minimal.  
   - The **interquartile range (IQR)** is also similar, indicating that most R² values are concentrated around high values (**above 0.92**).  

5. **Maximum Value**:  
   - The max R² values are nearly identical (**0.9989 vs. 0.9993**), meaning that for the best-fit cases, the difference in data selection method has no significant effect.  

### Interpretation:
- **Removing discontinuous periods seems to slightly reduce variability** (lower standard deviation).  
- **Extremely low R² values disappear** in the continuous dataset, suggesting that missing data might be **hurting model performance** for some stations.  
- **For the majority of stations**, there’s no significant difference in R², meaning that the **baseflow recession analysis is robust** whether or not small gaps in data exist.  

### Next Steps:
- You could **investigate the stations where R² dropped significantly** (e.g., check which stations had an R² < 0.5 in the ALL_DATA case but improved in CONT_DATA).

In [18]:
#### Step 1: Check Normality of Differences ####
# First, test if the differences are normally distributed using the Shapiro-Wilk test.
# If p > 0.05, the differences follow a normal distribution, and you can use a paired t-test.
# If p ≤ 0.05, the differences do not follow a normal distribution, and you should use the Wilcoxon signed-rank test instead.

# Compute the differences
differences = df_results["R_sq_with_weights"] - df_results2["R_sq_with_weights"]
differences = differences.dropna()  # Remove NaN values

# Shapiro-Wilk test for normality
shapiro_test = stats.shapiro(differences)
print("Shapiro-Wilk test p-value:", shapiro_test.pvalue)

In [19]:
#### Step 2: Perform Statistical Test ####

# Drop NaN values before performing the test
valid_data = df_results["R_sq_with_weights"].dropna()
valid_cont_data = df_results2["R_sq_with_weights"].dropna()

# Ensure both series have the same length
valid_data, valid_cont_data = valid_data.align(valid_cont_data, join='inner')

# A) If Differences Are Normally Distributed → Use Paired t-test
#t_stat, p_value = stats.ttest_rel(valid_data, valid_cont_data)
#print("Paired t-test p-value:", p_value)

# B) If Differences Are Not Normally Distributed → Use Wilcoxon Signed-Rank Test
wilcoxon_test = stats.wilcoxon(valid_data, valid_cont_data)
print("Wilcoxon test p-value:", wilcoxon_test.pvalue)

- **If p ≤ 0.05**, the difference is statistically significant.
- **If p > 0.05**, there is no significant difference.

---

### **Interpreting the Results**
- If **both tests give p > 0.05**, there is **no significant difference** in R² values between the two datasets.
- If **p ≤ 0.05**, the difference is **statistically significant**, meaning using the longest continuous period **impacts the R² values**.

**CONCLUSION:** Based on the analysis, it is better to use the results obtained with the longest continuous data periods (`df_results2`). One key point is that by considering these results, some very poor model fits are avoided and the selected alpha values are those that were determined from model fits with $R^2$ greater than 0.5.

In [20]:
k_recessions = df_results2[["station", "alpha_with_weights"]].copy()
k_recessions["k_recession"] = k_recessions["alpha_with_weights"].apply(lambda row: np.exp(-row))
k_recessions = k_recessions.dropna().reset_index(drop=True)
k_recessions.to_csv(lcd_folder + "//k_recessions_df.csv", index=False) # Save file for possible further analysis
k_recessions